In [49]:
import pandas as pd
import numpy as np
import pyarrow as pa
from sas7bdat import SAS7BDAT
import os, sys
from dotenv import load_dotenv

In [50]:
# Load environment variables

load_dotenv()
OUTPUT_DEMO_PATH = os.environ.get("OUTPUT_DEMO_PATH")
INPUT_PATH = os.environ.get("INPUT_PATH")

In [51]:
# Peek at the demo output to verify results at the end
output_demo = pd.read_parquet(
    OUTPUT_DEMO_PATH,
    engine="pyarrow",
)

output_demo.head()

time_index  number_of_trades              first_trade_time  \
ticker date                                                                     
AAPL   2023-08-18           0                26 2023-08-18 09:30:00.002929632   
       2023-08-18           1                86 2023-08-18 09:30:00.130872734   
       2023-08-18           2                61 2023-08-18 09:30:00.232653391   
       2023-08-18           3                77 2023-08-18 09:30:00.441937620   
       2023-08-18           4                 3 2023-08-18 09:30:00.568664722   

                                last_trade_time    open     high     low  \
ticker date                                                                
AAPL   2023-08-18 2023-08-18 09:30:00.130151264  172.30  172.310  172.19   
       2023-08-18 2023-08-18 09:30:00.232622254  172.31  172.310  172.13   
       2023-08-18 2023-08-18 09:30:00.441489008  172.14  172.340  172.06   
       2023-08-18 2023-08-18 09:30:00.568380810  172.12  172.235  172.00   
       2023-08-18 2023-08-18 09:30:00.568723916  172.02  172.020  172.00   

                    close     volume  
ticker date                           
AAPL   2023-08-18  172.23     5929.0  
       2023-08-18  172.13     5839.0  
       2023-08-18  172.10     5824.0  
       2023-08-18  172.00  2040137.0  
       2023-08-18  172.00  2037108.0

In [52]:
IDENTIFIERS = {"AAPL", "TSLA", "BRK"} # Only SYM_ROOTs are stored. Since we have only 1 SYM_SUFFIX, we will not explicity store it.

In [53]:
filtered_rows = []
HEADER = [
    "DATE",
    "TIME_M",
    "SYM_ROOT",
    "SYM_SUFFIX",
    "type",
    "EX",
    "TR_SCOND",
    "SIZE",
    "PRICE",
    "TR_SEQNUM",
    "QTime",
    "NBO",
    "NBB",
    "NBOqty",
    "NBBqty",
]
SYM_ROOT = HEADER.index("SYM_ROOT")
SYM_SUFFIX = HEADER.index("SYM_SUFFIX")

with SAS7BDAT(INPUT_PATH) as f:
    # Read the data in chunks. This prevents loading the entire dataset into memory.
    for row in f:
        if (
            (row[SYM_ROOT] == "BRK" and row[SYM_SUFFIX] == "A")
            or row[SYM_ROOT] == "AAPL"
            or row[SYM_ROOT] == "TSLA"
        ):
            filtered_rows.append(row)

df = pd.DataFrame(filtered_rows, columns=HEADER)

df.head()

[input.sas7bdat] header length 65536 != 8192
[input.sas7bdat] [input.sas7bdat] header length 65536 != 8192
[input.sas7bdat] [input.sas7bdat] [input.sas7bdat] header length 65536 != 8192
[input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] header length 65536 != 8192
[input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] header length 65536 != 8192
[input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] header length 65536 != 8192
[input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] header length 65536 != 8192
[input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] [input.sas7bdat] header length 65536 != 8192


,DATE,TIME_M,SYM_ROOT,SYM_SUFFIX,type,EX,TR_SCOND,SIZE,PRICE,TR_SEQNUM,QTime,NBO,NBB,NBOqty,NBBqty
0,23240.0,04:00:00.006787,AAPL,,T,K,@ TI,5.0,174.13,1829.0,04:00:00.006749,174.61,173.00,100.0,200.0
1,23240.0,04:00:00.008268,AAPL,,T,K,@ TI,10.0,174.13,1830.0,04:00:00.007828,174.61,173.02,100.0,100.0
2,23240.0,04:00:00.010602,AAPL,,T,K,@ TI,1.0,174.01,1831.0,04:00:00.007828,174.61,173.02,100.0,100.0
3,23240.0,04:00:00.011742,AAPL,,T,K,@ TI,5.0,173.45,1832.0,04:00:00.010747,174.50,173.02,100.0,100.0
4,23240.0,04:00:00.013349,AAPL,,T,K,@ TI,1.0,173.76,1833.0,04:00:00.013212,174.13,173.10,100.0,100.0


In [54]:
df.shape[0]

2059591

In [55]:
THRESHOLD = 1000000 # USD

In [56]:
# Concatenate the SYM_ROOT and SYM_SUFFIX

df["ticker"] = df.apply(
    lambda row: (
        f"{row['SYM_ROOT']}.{row['SYM_SUFFIX']}"
        if row["SYM_SUFFIX"]
        else row["SYM_ROOT"]
    ),
    axis=1,
)

df.head()

,DATE,TIME_M,SYM_ROOT,SYM_SUFFIX,type,EX,TR_SCOND,SIZE,PRICE,TR_SEQNUM,QTime,NBO,NBB,NBOqty,NBBqty,ticker
0,23240.0,04:00:00.006787,AAPL,,T,K,@ TI,5.0,174.13,1829.0,04:00:00.006749,174.61,173.00,100.0,200.0,AAPL
1,23240.0,04:00:00.008268,AAPL,,T,K,@ TI,10.0,174.13,1830.0,04:00:00.007828,174.61,173.02,100.0,100.0,AAPL
2,23240.0,04:00:00.010602,AAPL,,T,K,@ TI,1.0,174.01,1831.0,04:00:00.007828,174.61,173.02,100.0,100.0,AAPL
3,23240.0,04:00:00.011742,AAPL,,T,K,@ TI,5.0,173.45,1832.0,04:00:00.010747,174.50,173.02,100.0,100.0,AAPL
4,23240.0,04:00:00.013349,AAPL,,T,K,@ TI,1.0,173.76,1833.0,04:00:00.013212,174.13,173.10,100.0,100.0,AAPL


In [57]:
# Preserve only the relevant rows

df = df[["DATE", "TIME_M", "ticker", "SIZE", "PRICE"]]

df.head()

,DATE,TIME_M,ticker,SIZE,PRICE
0,23240.0,04:00:00.006787,AAPL,5.0,174.13
1,23240.0,04:00:00.008268,AAPL,10.0,174.13
2,23240.0,04:00:00.010602,AAPL,1.0,174.01
3,23240.0,04:00:00.011742,AAPL,5.0,173.45
4,23240.0,04:00:00.013349,AAPL,1.0,173.76


In [58]:
# Reformat the time column

df["DATE"] = pd.to_datetime(df["DATE"], unit="D", origin="1960-01-01")

df.head()

,DATE,TIME_M,ticker,SIZE,PRICE
0,2023-08-18,04:00:00.006787,AAPL,5.0,174.13
1,2023-08-18,04:00:00.008268,AAPL,10.0,174.13
2,2023-08-18,04:00:00.010602,AAPL,1.0,174.01
3,2023-08-18,04:00:00.011742,AAPL,5.0,173.45
4,2023-08-18,04:00:00.013349,AAPL,1.0,173.76


In [70]:
# Filter the times to only include those within the stock market opening hours

filtered_df = df[
    (df["TIME_M"] >= pd.to_datetime("09:30").time())
    & (df["TIME_M"] <= pd.to_datetime("16:00").time())
]

filtered_df.head()

,DATE,TIME_M,ticker,SIZE,PRICE
25760,2023-08-18,09:30:00.002930,AAPL,300.0,172.30
25761,2023-08-18,09:30:00.011063,AAPL,400.0,172.23
25762,2023-08-18,09:30:00.015240,AAPL,500.0,172.23
25763,2023-08-18,09:30:00.030387,AAPL,30.0,172.23
25764,2023-08-18,09:30:00.052547,AAPL,200.0,172.24
